### Q1. You are working on a machine learning project where you have a dataset containing numerical and categorical features. You have identified that some of the features are highly correlated and there are missing values in some of the columns. You want to build a pipeline that automates the feature engineering process and handles the missing values.
Design a pipeline that includes the following steps:
    
* Use an automated feature selection method to identify the important features in the dataset.
* Create a numerical pipeline that includes the following steps.
* Impute the missing values in the numerical columns using the mean of the column values.
* Scale the numerical columns using standardisation.
* Create a categorical pipeline that includes the following steps.
* Impute the missing values in the categorical columns using the most frequent value of the column.
* One-hot encode the categorical columns.
* Combine the numerical and categorical pipelines using a ColumnTransformer.
* Use a Random Forest Classifier to build the final model.
* Evaluate the accuracy of the model on the test dataset.

Note! Your solution should include code snippets for each step of the pipeline, and a brief explanation of each step. You should also provide an interpretation of the results and suggest possible improvements for the pipeline.

In [97]:
# Importing necessary libraries

import seaborn as sns

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import VotingClassifier

from sklearn.metrics import accuracy_score

In [84]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [85]:
X = iris.drop('species',axis=1)
y = iris['species']
X.shape, y.shape

((150, 4), (150,))

In [86]:
le = LabelEncoder()
y = le.fit_transform(y)


In [87]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.3,
                                                    random_state=123)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((105, 4), (45, 4), (105,), (45,))

In [88]:
cat_col = []
num_col = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

In [89]:
num_pipeline = Pipeline(
        steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ])

cat_pipeline = Pipeline(
        steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehotencoder',OneHotEncoder())
        ])

In [90]:
preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, num_col),
    ('col_pipeline', cat_pipeline, cat_col)
])

In [78]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [55]:
rf = RandomForestClassifier(random_state=123)

In [56]:
rf.fit(X_train,y_train)

RandomForestClassifier(random_state=123)

In [58]:
y_pred = rf.predict(X_test)

In [60]:
print(accuracy_score(y_test,y_pred))

0.9333333333333333


Possible improvements for this pipeline could include:

* Trying different feature selection methods to see which one works best for this dataset.
* Trying different imputation strategies to see if they improve the model's performance.
* Trying different scaling methods to see if they improve the model's performance.
* Trying different models to see if they improve the model's performance.

### Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.

In [102]:
models = [('Random Forest', RandomForestClassifier()),
          ('Logistic Regression', LogisticRegression())]
voting_classifier = VotingClassifier(models, voting='hard')


In [103]:
def evaluate_model(X_train,y_train,X_test,y_test,voting_classifier):
    
    report = {}
    voting_classifier.fit(X_train, y_train)
    y_test_pred = voting_classifier.predict(X_test)
    test_model_score = accuracy_score(y_test, y_test_pred)
    report = {'Voting Classifier': test_model_score}
    return report

In [104]:
evaluate_model(X_train,y_train,X_test,y_test,voting_classifier)

{'Voting Classifier': 0.9333333333333333}